In [1]:
import numpy as np
np.set_printoptions(precision=4, suppress=True)

In [2]:
F = 4500 # N
v = 3.05 # m/s
D = 500 # mm
L = 4 # year
t1 = 12 # s
t2 = 60 # s
T1 = 1 # N.mm
T2 = 0.7 # N.mm

## Motor Design $(power, rpm)$

In [3]:
# calculate efficiency, e = eta
e_coupling = 1
e_bearing = 0.99
e_helical_gear = 0.96
e_chain = 0.95
e_sys = e_coupling * e_bearing**3 * e_helical_gear * e_chain
print(e_sys)

0.8849126879999999


**Find theoretical power of motor**

In [4]:
P_max = F*v/1000 # calculate max power of the belt conveyor, kW
P_work = P_max*np.sqrt((T1**2*t1 + T2**2*t2)/(t1+t2)) # calculate power given the work load, kW
P_motor = P_work/e_sys # lower limit of the motor's power, kW
print(P_max, P_work, P_motor)

13.725 10.407496546960752 11.761043420546766


**Find theoretical rpm of motor**

In [5]:
n_belt = v*6e4/np.pi/D # calculate rotational speed of the belt conveyor, rpm
u_chain = 5
u_helical_gear = 5
u_sys = u_chain * u_helical_gear # calculate transmission ratio of the system
n_motor = u_sys * n_belt # calculate theoretical rotational speed of the motor, rpm
print(n_belt, u_sys, n_motor)

116.50141834326739 25 2912.535458581685


**From table P1.3, p.236, choose motor 4A160S2Y3**

In [6]:
P_motor = 15 # power based on choice of motor, kW
n_motor = 2930 # rotational speed based on choice of motor, rpm
u_sys = n_motor / n_belt # new transmission ratio
u_chain = u_sys / u_helical_gear # new transmission ratio of chain drive, given u_helical_gear=const
print(u_sys, u_chain)

25.14990840168878 5.029981680337756


**Calculate power, rpm and torque of the motor and 2 shafts**

In [7]:
P_chain = P_work
P_shaft_2 = P_chain / (e_chain) # power of shaft 2, kW
P_shaft_1 = P_shaft_2 / (e_bearing*e_helical_gear) # power of shaft 1, kW
P_motor = P_shaft_1 / (e_bearing*e_coupling) # power of motor, kW
print(P_motor, P_shaft_1, P_shaft_2)

11.643432986341297 11.526998656477884 10.955259523116581


In [8]:
n_shaft_1 = n_motor # rotational speed of shaft 1, rpm
n_shaft_2 = n_shaft_1 / (u_helical_gear) # rotational speed of shaft 2, rpm
print(n_motor, n_shaft_1, n_shaft_2)

2930 2930 586.0


In [9]:
T_motor = 9.55e6 * P_motor / n_motor # torque of motor, N.mm
T_shaft_1 = 9.55e6 * P_shaft_1 / n_shaft_1 # torque of shaft 1, N.mm
T_shaft_2 = 9.55e6 * P_shaft_2 / n_shaft_2 # torque of shaft 2, N.mm
print(T_motor, T_shaft_1, T_shaft_2)

37950.43857322846 37570.934187496176 178537.07925898183


## Chain Drive Design

**Find pitch $p$**

In [10]:
# find z1, z2
z_1 = np.ceil(29 - 2*u_chain)
z_2 = np.ceil(u_chain*z_1) + 1 # eq (5.1), p.80, plus 1 to obtain odd number of teeth

# find k, k_z, k_n
n01 = 600 # ~ n_shaft_2
k_z, k_n = 25/z_1, n01/n_shaft_2 # eq (5.3), p.81
k_0, k_a, k_dc, k_bt, k_d, k_c = 1, 1, 1, 1, 1.25, 1.3
k = k_0 * k_a * k_dc * k_bt * k_d * k_c # table 5.6, p.82
P_allowance = P_chain * k * k_z * k_n # equals 22.7845 <= 25.7
print(P_allowance)
P_allowance, p, dc, B = 25.7, 25.4, 7.95, 22.61 # table 5.5, p.81
print(z_1, z_2, P_allowance, p) # examine table 5.8, p<p_max~31.75

22.78450946040671
19.0 97.0 25.7 25.4


In [11]:
d_1 = p/np.sin(np.pi/z_1)
d_2 = p/np.sin(np.pi/z_2)
print(d_1,d_2)

154.3185590527462 784.389021668636


**Find center distance $a$ and number of chain $x_c$**

In [12]:
a_min, a_max, a = 30*p, 50*p, 770 # 762<a<1270, choose a = 800
x = 2*a/p + (z_1+z_2)/2 + (z_2-z_1)**2*p/(4*np.pi**2*a) # eq (5.12), p.85
x_c = np.ceil(x)
a = 0.25*p*(x_c-0.5*(z_2+z_1)+np.sqrt((x_c-0.5*(z_2+z_1))**2-2*((z_2-z_1)/np.pi)**2)) - 0.003*a # eq (5.13), p.85
i = z_1*n_shaft_2/(15*x) # eq (5.14), p.85; i<[i]=30 at table 5.9, p.85

**Validate strength of the chain drive**

In [13]:
B_min, d_O, d_l, h_max, b_max, Q, q = 15.88, 7.95, 15.88, 24.2, 38, 56.7, 2.6 # table 5.2, p.78
k_f, k_d = 6, 1.2
v_1 = n_shaft_2*p*z_1/6e4
F_t, F_v, F_O = 1e3*P_chain/v_1, q*v_1**2, 9.81*k_f*q*a/1e3
s = Q*1e3/(k_d*F_t + F_O + F_v)
s >= 10.3 # table 5.10, p.86

True

In [14]:
k_x = 1.15
F_r = k_x*F_t # eq (5.20), p.88
print(F_r)

2539.279067664951


In [18]:
alp, beta = 20*np.pi/180, 20*np.pi/180
H1, H2, NFO1, NFO2 = 250, 240, 4e6,4e6
NHO1, NHO2 = 30*H1**2.4, 30*H2**2.4
NHE1, NHE2 = 60*n_shaft_1*(T1**3*t1 + T2**3*t2), 60*n_shaft_2*(T1**3*t1 + T2**3*t2)
NFE1, NFE2 = 60*n_shaft_1*(T1**6*t1 + T2**6*t2), 60*n_shaft_2*(T1**6*t1 + T2**6*t2)
KHL1, KHL2 = (NHO1/NHE1)**(1/6), (NHO2/NHE2)**(1/6)
KFL1, KFL2 = (NFO1/NFE1)**(1/6), (NFO2/NFE2)**(1/6)
sigH1, sigH2 = 570/1.1*KHL1, 550/1.1*KHL2
sigF1, sigF2 = 450*KFL1/1.75, 432*KFL2/1.75
sigH = np.average([sigH1,sigH2])
psibd = 0.53*0.5*(u_helical_gear+1)
da1 = 43*(u_helical_gear+1)*(T_shaft_1*1.108/sigH**2/u_helical_gear/0.5)**(1/3)

alp_t = np.arctan(np.tan(alp)/np.cos(beta))
beta_b = np.arctan(np.cos(alp_t)*np.tan(beta))
zH = np.sqrt(2*np.cos(beta_b)/np.sin(2*alp_t))

In [21]:
np.round([alp_t*180/np.pi, beta_b*180/np.pi, zH],2)

array([21.17, 18.75,  1.68])